In [1]:
import csv, os, numpy as np, datetime as dt

# ------------------ FALSIFIABILITY FUNCTIONS (embedded) ------------------
def TET_falsifiable(CMB_B_modes, galaxy_excess, echo_delay, sys_err=0.01):
    # Contrôle topologique
    if galaxy_excess < 3.0:
        return "REJETÉ (violation conservation Φ)"
    # Fenêtres prévues (placeholders d'après votre note)
    if (37 - sys_err <= CMB_B_modes <= 47 + sys_err) and \
       (galaxy_excess >= 4.3) and \
       (16.5e-3 - sys_err <= echo_delay <= 18.5e-3 + sys_err):
        return "NON FALSIFIÉ"
    return "REJETÉ"

def TUTT_falsifiable(BAO_anisotropy, CMB_cross_corr, photon_disp, sys_err=0.005):
    # Contrôle covariance
    if photon_disp > 1e-22:
        return "REJETÉ (violation covariance)"
    # Fenêtres prévues (placeholders d'après votre note)
    if (0.0077 - sys_err <= BAO_anisotropy <= 0.0087 + sys_err) and \
       (CMB_cross_corr >= 7.8e-9) and \
       (photon_disp < 2.5e-22):
        return "NON FALSIFIÉ"
    return "REJETÉ"

def TEM_falsifiable(echo_delay, delta_H0, photon_disp, sys_err=0.001):
    # Contrôle CPT
    if photon_disp > 1e-23:
        return "REJETÉ (violation CPT potentielle)"
    # Fenêtres prévues (placeholders d'après votre note)
    if (16.4e-3 - sys_err <= echo_delay <= 18.0e-3 + sys_err) and \
       (0.008 - sys_err <= delta_H0 <= 0.012 + sys_err) and \
       (photon_disp < 1.5e-24):
        return "NON FALSIFIÉ"
    return "REJETÉ"


DATA_DIR = "data"  # Assuming data is in a 'data' subdirectory
REPORT_DIR = "report" # Assuming reports go in a 'report' subdirectory
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(REPORT_DIR, exist_ok=True)

def load_cmb_demo():
    # CMB B-modes (une valeur agrégée + sigma)
    # Démo: 41 +/- 6 nK^2 (compatible TET demo)
    return {"Dl_BB": 41.0, "sigma": 6.0, "galaxy_excess": 4.9}

def load_bao_demo():
    # BAO anisotropy (δθ/θ), cross-corr (C_ell^TE), photon dispersion
    # Démo: valeurs proches de celles listées dans vos notes
    return {"BAO_anisotropy": 0.0082, "CMB_cross_corr": 8.9e-9, "photon_disp": 2.0e-22}

def load_gw_demo():
    # GW echoes (Δt) + params auxiliaires
    return {"echo_delay": 17.3e-3, "sigma": 0.0004, "delta_H0": 0.0097}

def try_load_csvs(base_dir):
    data = {}
    # Optional: user can replace demo by actual csv files
    try:
        with open(os.path.join(base_dir, "cmb.csv")) as f:
            r = list(csv.DictReader(f))
            if r:
                row = r[0]
                data["Dl_BB"] = float(row.get("Dl_BB", "41"))
                data["cmb_sigma"] = float(row.get("sigma", "6"))
                data["galaxy_excess"] = float(row.get("galaxy_excess", "4.9"))
    except Exception:
        d = load_cmb_demo()
        data.update({"Dl_BB": d["Dl_BB"], "cmb_sigma": d["sigma"], "galaxy_excess": d["galaxy_excess"]})
    try:
        with open(os.path.join(base_dir, "bao.csv")) as f:
            r = list(csv.DictReader(f))
            if r:
                row = r[0]
                data["BAO_anisotropy"] = float(row.get("BAO_anisotropy", "0.0082"))
                data["CMB_cross_corr"] = float(row.get("CMB_cross_corr", "8.9e-9"))
                data["photon_disp"] = float(row.get("photon_disp", "2.0e-22"))
    except Exception:
        d = load_bao_demo()
        data.update(d)
    try:
        with open(os.path.join(base_dir, "gw.csv")) as f:
            r = list(csv.DictReader(f))
            if r:
                row = r[0]
                data["echo_delay"] = float(row.get("echo_delay", "0.0173"))
                data["gw_sigma"] = float(row.get("sigma", "0.0004"))
                data["delta_H0"] = float(row.get("delta_H0", "0.0097"))
    except Exception:
        d = load_gw_demo()
        data.update({"echo_delay": d["echo_delay"], "gw_sigma": d["sigma"], "delta_H0": d["delta_H0"]})
    return data


if __name__ == "__main__":
    X = try_load_csvs(DATA_DIR)
    # Falsifiability verdicts
    tet_v = TET_falsifiable(X["Dl_BB"], X["galaxy_excess"], X["echo_delay"])
    tutt_v = TUTT_falsifiable(X["BAO_anisotropy"], X["CMB_cross_corr"], X["photon_disp"])
    tem_v = TEM_falsifiable(X["echo_delay"], X["delta_H0"], X["photon_disp"])

    # Summary
    now = dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
    lines = []
    lines.append("=== H++ Multi-Theory Comparison ===")
    lines.append(f"Timestamp: {now}")
    lines.append("")
    lines.append("[Inputs]")
    for k in ["Dl_BB","galaxy_excess","echo_delay","BAO_anisotropy","CMB_cross_corr","photon_disp","delta_H0"]:
        if k in X:
            lines.append(f"  - {k}: {X[k]}")
    lines.append("")
    lines.append("[Verdicts]")
    lines.append(f"  - TET : {tet_v}")
    lines.append(f"  - TUTT: {tutt_v}")
    lines.append(f"  - TEM : {tem_v}")
    lines.append("")
    # Simple policy
    if [tet_v, tutt_v, tem_v].count("NON FALSIFIÉ") == 0:
        lines.append("=> Aucune des 3 théories n'est soutenue par ces entrées (toutes REJETÉES).")
    else:
        ok = [n for n,v in zip(['TET','TUTT','TEM'], [tet_v,tutt_v,tem_v]) if v=='NON FALSIFIÉ']
        lines.append(f"=> Théories compatibles pour ces entrées: {', '.join(ok)}")

    report_path = os.path.join(REPORT_DIR, "verdict.txt")
    with open(report_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))
    print("\n".join(lines))
    print(f"\nRapport sauvegardé: {report_path}")

ModuleNotFoundError: No module named 'sim'

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
H++ ONE-FILE COMPARATOR (TET / TUTT / TEM)
------------------------------------------
Usage (local/Colab):
    python hpp_onefile_compare.py

Options:
- Place optional CSVs in the SAME FOLDER as this script:
  - cmb.csv with columns: Dl_BB,sigma,galaxy_excess
  - bao.csv with columns: BAO_anisotropy,CMB_cross_corr,photon_disp
  - gw.csv  with columns: echo_delay,delta_H0,sigma

If a CSV is missing, demo values are used.
A text report is saved next to the script: report_verdict.txt
"""

import os, csv, datetime as dt

# ------------------ DATA LOADERS (CSV or demo) ------------------
def _first_row(path, defaults):
    try:
        with open(path) as f:
            r = list(csv.DictReader(f))
            if r:
                return {k: float(r[0].get(k, defaults[k])) for k in defaults}
    except Exception:
        pass
    return defaults.copy()

def load_inputs(base_dir):
    cmb = _first_row(os.path.join(base_dir, "cmb.csv"),
                     {"Dl_BB":41.0, "sigma":6.0, "galaxy_excess":4.9})
    bao = _first_row(os.path.join(base_dir, "bao.csv"),
                     {"BAO_anisotropy":0.0082, "CMB_cross_corr":8.9e-9, "photon_disp":2.0e-22})
    gw  = _first_row(os.path.join(base_dir, "gw.csv"),
                     {"echo_delay":0.0173, "delta_H0":0.0097, "sigma":0.0004})
    X = {}
    X.update({"Dl_BB": cmb["Dl_BB"], "galaxy_excess": cmb["galaxy_excess"]})
    X.update({"BAO_anisotropy": bao["BAO_anisotropy"], "CMB_cross_corr": bao["CMB_cross_corr"], "photon_disp": bao["photon_disp"]})
    X.update({"echo_delay": gw["echo_delay"], "delta_H0": gw["delta_H0"]})
    return X

# ------------------ MAIN ------------------
def main():
    base = "." # Use the current directory as the base
    X = load_inputs(base)
    # Verdicts
    tet_v  = TET_falsifiable(X["Dl_BB"], X["galaxy_excess"], X["echo_delay"])
    tutt_v = TUTT_falsifiable(X["BAO_anisotropy"], X["CMB_cross_corr"], X["photon_disp"])
    tem_v  = TEM_falsifiable(X["echo_delay"], X["delta_H0"], X["photon_disp"])

    now = dt.datetime.utcnow().strftime("%Y-%m-%d %H:%M:%S UTC")
    lines = []
    lines.append("=== H++ ONE-FILE COMPARISON ===")
    lines.append(f"Timestamp: {now}")
    lines.append("")
    lines.append("[Inputs]")
    for k in ["Dl_BB","galaxy_excess","echo_delay","BAO_anisotropy","CMB_cross_corr","photon_disp","delta_H0"]:
        lines.append(f"  - {k}: {X[k]}")
    lines.append("")
    lines.append("[Verdicts]")
    lines.append(f"  - TET : {tet_v}")
    lines.append(f"  - TUTT: {tutt_v}")
    lines.append(f"  - TEM : {tem_v}")
    lines.append("")
    if [tet_v, tutt_v, tem_v].count("NON FALSIFIÉ") == 0:
        lines.append("=> Aucune des 3 théories n'est soutenue par ces entrées (toutes REJETÉES).")
    else:
        ok = [n for n,v in zip(['TET','TUTT','TEM'], [tet_v,tutt_v,tem_v]) if v=='NON FALSIFIÉ']
        lines.append(f"=> Théories compatibles pour ces entrées: {', '.join(ok)}")

    report_path = os.path.join(base, "report_verdict.txt")
    with open(report_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    print("\n".join(lines))
    print(f"\nRapport sauvegardé: {report_path}")

if __name__ == "__main__":
    main()

NameError: name '__file__' is not defined